In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:

!pip install keras==2.1.6


In [15]:
import os
import pandas as pd
import glob
import cv2
import shutil
from tqdm import tqdm
import keras


In [4]:
KAGGLE_PATH=os.path.expanduser("~/.kaggle")


In [5]:
PATH = KAGGLE_PATH + '/competitions/dog-breed-identification'


In [6]:
labels_df = pd.read_csv(PATH+"/labels.csv")
labels_df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [7]:
dest_folder = "train_folder"
shutil.rmtree(f'{PATH}/{dest_folder}',ignore_errors=True)
os.makedirs(f'{PATH}/{dest_folder}', exist_ok=True)


for breed in labels_df['breed'].unique():
    os.makedirs(f'{PATH}/{dest_folder}/{breed}', exist_ok=True)

for f, breed in tqdm(labels_df.values):
    original_file = f'{PATH}/train/{f}.jpg'
    os.makedirs(f'{PATH}/{dest_folder}', exist_ok=True)
    destination_file= f'{PATH}/{dest_folder}/{breed}/{f}.jpg'
    shutil.copyfile(original_file, destination_file)


100%|██████████| 10222/10222 [00:01<00:00, 8002.36it/s]


In [16]:
from keras.preprocessing.image import ImageDataGenerator


import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50
from keras.applications.vgg19 import preprocess_input as preprocess_input_vgg19

from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from keras import metrics

from keras.applications.vgg19 import VGG19


In [17]:
TRAINING_DIR=f'{PATH}/{dest_folder}'
IMAGE_SIZE=224
BATCH_SIZE=64
NUM_CLASSES=120

In [18]:
#!ls {TRAINING_DIR}

In [ ]:
?ImageDataGenerator

In [19]:
data_generator  = ImageDataGenerator(preprocessing_function=preprocess_input_resnet50,
    
                                     rotation_range=45,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.25,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                    validation_split=0.20
                                    )

train_generator = data_generator.flow_from_directory(TRAINING_DIR,  shuffle=True, seed=13,
                                                     target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="training")

validation_generator = data_generator.flow_from_directory(TRAINING_DIR, target_size=(IMAGE_SIZE, IMAGE_SIZE), seed=13,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="validation")


Found 8221 images belonging to 120 classes.
Found 2001 images belonging to 120 classes.


In [20]:
#base_model = ResNet50(weights='imagenet', include_top=False)
base_model = ResNet50(weights='imagenet', include_top=False)


x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
#x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
#x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [21]:
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers: layer.trainable = False
optimizer = keras.optimizers.RMSprop(lr=1e-2)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [23]:
model_callbacks = [EarlyStopping(monitor='val_acc', patience=5, verbose=1)
                   , ReduceLROnPlateau(monitor='val_acc', factor=0.5, verbose=1,
                                     patience=3, min_lr=1e-2)
                  , ModelCheckpoint(filepath='weights.best.mymodel.hdf5', 
                               verbose=1, save_best_only=True)
                  
                  ]

In [ ]:
train_generator.n // BATCH_SIZE

In [ ]:
?model.fit_generator

In [24]:
model.fit_generator(train_generator, train_generator.n // BATCH_SIZE, epochs=15, workers=4,
                    callbacks=model_callbacks,
                    use_multiprocessing=True,
        validation_data=validation_generator, validation_steps=validation_generator.n // BATCH_SIZE)




Epoch 1/15
128/128 [==============================] - 94s 732ms/step - loss: 15.8887 - acc: 0.0088 - val_loss: 16.0125 - val_acc: 0.0066

Epoch 00001: val_loss improved from inf to 16.01248, saving model to weights.best.mymodel.hdf5
Epoch 2/15
128/128 [==============================] - 86s 673ms/step - loss: 16.0079 - acc: 0.0068 - val_loss: 16.0125 - val_acc: 0.0066

Epoch 00002: val_loss did not improve from 16.01248
Epoch 3/15
128/128 [==============================] - 87s 681ms/step - loss: 16.0118 - acc: 0.0066 - val_loss: 16.0125 - val_acc: 0.0066

Epoch 00003: val_loss did not improve from 16.01248
Epoch 4/15
128/128 [==============================] - 87s 682ms/step - loss: 16.0079 - acc: 0.0068 - val_loss: 16.0125 - val_acc: 0.0066

Epoch 00004: val_loss did not improve from 16.01248
Epoch 5/15
128/128 [==============================] - 87s 682ms/step - loss: 16.0059 - acc: 0.0070 - val_loss: 16.0125 - val_acc: 0.0066

Epoch 00005: val_loss did not improve from 16.01248
Epoch 6

In [ ]:
k

In [ ]:
split_at = 140
for layer in model.layers[:split_at]: layer.trainable = False
for layer in model.layers[split_at:]: layer.trainable = True
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator, train_generator.n // BATCH_SIZE, epochs=10, workers=4,
                    callbacks=model_callbacks,
        validation_data=validation_generator, validation_steps=validation_generator.n // BATCH_SIZE)


